<a href="https://colab.research.google.com/github/MatiasSingermann/IBODS-AI/blob/main/IA_CLEAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importacion e instalacion de librerias

In [ ]:
!pip install roboflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached urllib3-1.26.6-py2.py3-none-any.whl (138 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tflite-model-maker 0.4.2 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.6 which is incompatible.


In [ ]:
!pip install tflite-model-maker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.6
    Uninstalling urllib3-1.26.6:
      Successfully uninstalled urllib3-1.26.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
roboflow 0.2.20 requires urllib3==1.26.6, but you have urllib3 1.25.11 which is incompatible.


Importo `Tensorflow` y varios de sus **modulos**.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Input
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tflite_model_maker import object_detector
import tensorflow_hub as hub
from tflite_model_maker.config import QuantizationConfig
from tflite_support.task import vision
from tflite_support.task.vision import ObjectDetector
from tflite_support.task.vision import ObjectDetectorOptions
from tflite_support.task.processor import DetectionOptions
from tflite_support.task.core import BaseOptions

In [ ]:
!pip list

Package                       Version
----------------------------- ----------------------
absl-py                       1.3.0
aeppl                         0.0.33
aesara                        2.7.9
aiohttp                       3.8.3
aiosignal                     1.3.1
alabaster                     0.7.12
albumentations                1.2.1
altair                        4.2.0
appdirs                       1.4.4
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-timeout                 4.0.2
asynctest                     0.13.0
atari-py                      0.2.9
atomicwrites                  1.4.1
attrs                         22.1.0
audioread                     3.0.0
autograd                      1.5
Babel                         2.11.0
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        5.0.1
blis                          0.7.9
bokeh

In [ ]:
!python --version

Python 3.7.15


Importo varias `librerias` que me permiten trabajar con **tensores**, y **operaciones matematicas** necesrias. 

In [ ]:
import pandas as pd
import numpy as np
import torch
import torchvision.transforms as transforms
import operator as op
from collections import Counter

Importo `librerias` para **administrar archivos** y seleccionar de manera **random** alguno de ellos

In [ ]:
import os
import random

Importo `librerias` que me permiten trabajar con las **imagenes** del dataset.

In [ ]:
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from matplotlib.image import imread
from itertools import product
from skimage import draw, transform

Importo `Roboflow` para acceder al dataset

In [ ]:
from roboflow import Roboflow

## Importacion del **Dataset** y sus **labels**

Me conecto a mi cuenta de Drive para agarrar ciertos archivos

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Importo el dataset desde Roboflow

In [ ]:
rf = Roboflow(api_key="41l2d60Eq7cWhUOVgPxd")
project = rf.workspace("ibods").project("improved_ibods")
dataset = project.version(1).download("voc")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to IMPROVED_IBODS-1 in voc:: 100%|██████████| 41013/41013 [00:11<00:00, 3484.16it/s]


Asigno la **ruta del archivo** de las anotaciones a una variable

In [ ]:
path_lbl = '/content/gdrive/MyDrive/Proyecto 4to/I.A/Anotaciones/labels.txt'

Abro el **archivo de anotaciones** y lo *recorro linea por linea*, *eliminando los espacios* y metiendo los `strings` en una `lista`

In [ ]:
with open(path_lbl) as f:
   labels = [line.strip() for line in f.readlines()]
labels

['auto',
 'bicicleta',
 'cordon',
 'cruces',
 'cruzar',
 'escalones',
 'moto',
 'parar',
 'persona',
 'pozo']

Usando el **arhcivo de labels**, le asigno un *valor numerico* para que la `IA` pueda interpretar las anotaciones

In [ ]:
assigned_labels = { i : labels[i] for i in range(0, len(labels) ) }
assigned_labels

{0: 'auto',
 1: 'bicicleta',
 2: 'cordon',
 3: 'cruces',
 4: 'cruzar',
 5: 'escalones',
 6: 'moto',
 7: 'parar',
 8: 'persona',
 9: 'pozo'}

## Funciones Utiles y configuracion

In [ ]:
CLASSES = labels

In [ ]:
COLORS = np.random.randint(0, 255, size=(len(CLASSES), 3), dtype=np.uint8)

In [ ]:
def preprocess_image(image_path, input_size):
  ''' Preprocess the input image to feed to the TFLite model
  '''
  img = tf.io.read_file(image_path)
  img = tf.io.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.uint8)
  original_image = img
  resized_img = tf.image.resize(img, input_size)
  resized_img = resized_img[tf.newaxis, :]
  resized_img = tf.cast(resized_img, dtype=tf.uint8)
  return resized_img, original_image

In [ ]:
def detect_objects(interpreter, image, threshold):
  ''' Returns a list of detection results, each a dictionary of object info.
  '''

  signature_fn = interpreter.get_signature_runner()

  # Feed the input image to the model
  output = signature_fn(images=image)

  # Get all outputs from the model
  count = int(np.squeeze(output['output_0']))
  scores = np.squeeze(output['output_1'])
  classes = np.squeeze(output['output_2'])
  boxes = np.squeeze(output['output_3'])

  results = []
  for i in range(count):
    if scores[i] >= threshold:
      result = {
        'bounding_box': boxes[i],
        'class_id': classes[i],
        'score': scores[i]
      }
      results.append(result)
  return results

In [ ]:
def run_odt_and_draw_results(image_path, interpreter, threshold=0.5):
  ''' Run object detection on the input image and draw the detection results
  '''
  
  # Load the input shape required by the model
  _, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']

  # Load the input image and preprocess it
  preprocessed_image, original_image = preprocess_image(
      image_path,
      (input_height, input_width)
    )

  # Run object detection on the input image
  results = detect_objects(interpreter, preprocessed_image, threshold=threshold)

  # Plot the detection results on the input image
  original_image_np = original_image.numpy().astype(np.uint8)
  for obj in results:
    # Convert the object bounding box from relative coordinates to absolute
    # coordinates based on the original image resolution
    ymin, xmin, ymax, xmax = obj['bounding_box']
    xmin = int(xmin * original_image_np.shape[1])
    xmax = int(xmax * original_image_np.shape[1])
    ymin = int(ymin * original_image_np.shape[0])
    ymax = int(ymax * original_image_np.shape[0])

    # Find the class index of the current object
    class_id = int(obj['class_id'])

    # Draw the bounding box and label on the image
    color = [int(c) for c in COLORS[class_id]]
    cv2.rectangle(original_image_np, (xmin, ymin), (xmax, ymax), color, 2)
    # Make adjustments to make the label visible for all objects
    y = ymin - 15 if ymin - 15 > 15 else ymin + 15
    label = "{}: {:.0f}%".format(CLASSES[class_id], obj['score'] * 100)
    cv2.putText(original_image_np, label, (xmin, y),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

  # Return the final image
  original_uint8 = original_image_np.astype(np.uint8)
  return original_uint8

In [ ]:
def draw_bounding_box (image, model_path, threshold):
  base_opt = BaseOptions(model_path)

  detection_opt = DetectionOptions(score_threshold = threshold)

  options = ObjectDetectorOptions(base_opt, detection_opt)

  detector = ObjectDetector.create_from_options(options)

  tensor_image = vision.TensorImage.create_from_file(image)

  results = detector.detect(tensor_image)

  resultsdetect = results.detections

  for obj in resultsdetect:
    categories = obj.categories
    for category in categories:
      names = category.category_name
      scores = category.score * 100
    percentages = [float(scores)]
    for percentage in percentages:
      confidence = str(int(percentage)) + '%'
      label = names + ", " + confidence

  return label

## Cargar Dataset

### Cargo los datos de entrenamiento

In [ ]:
TRAIN_PATH = 'IMPROVED_IBODS-1/train'

In [ ]:
train_data = object_detector.DataLoader.from_pascal_voc(
    TRAIN_PATH,
    TRAIN_PATH,
    labels
)

### Cargo los datos de validacion

In [ ]:
VALID_PATH = 'IMPROVED_IBODS-1/valid'

In [ ]:
valid_data = object_detector.DataLoader.from_pascal_voc(
    VALID_PATH,
    VALID_PATH,
    labels
)

### Cargo los datos de testeo

In [ ]:
TEST_PATH = 'IMPROVED_IBODS-1/test'

In [ ]:
test_data = object_detector.DataLoader.from_pascal_voc(
    TEST_PATH,
    TEST_PATH,
    labels
)

## Entrenamiento de la IA

In [ ]:
model = object_detector.create(
    train_data,
    model_spec= 'efficientdet_lite3',
    batch_size= 8,
    train_whole_model=True,
    epochs=50,
    validation_data=valid_data
)

Epoch 1/50
2242/2242 [==============================] - 1720s 746ms/step - det_loss: 0.6260 - cls_loss: 0.4353 - box_loss: 0.0038 - reg_l2_loss: 0.0973 - loss: 0.7232 - learning_rate: 0.0090 - gradient_norm: 4.2451 - val_det_loss: 0.4315 - val_cls_loss: 0.2951 - val_box_loss: 0.0027 - val_reg_l2_loss: 0.0999 - val_loss: 0.5314
Epoch 2/50
2242/2242 [==============================] - 1680s 750ms/step - det_loss: 0.4491 - cls_loss: 0.3037 - box_loss: 0.0029 - reg_l2_loss: 0.1020 - loss: 0.5511 - learning_rate: 0.0100 - gradient_norm: 3.2948 - val_det_loss: 0.2647 - val_cls_loss: 0.1818 - val_box_loss: 0.0017 - val_reg_l2_loss: 0.1036 - val_loss: 0.3683
Epoch 3/50
2242/2242 [==============================] - 1676s 747ms/step - det_loss: 0.4035 - cls_loss: 0.2704 - box_loss: 0.0027 - reg_l2_loss: 0.1050 - loss: 0.5085 - learning_rate: 0.0099 - gradient_norm: 2.9496 - val_det_loss: 0.2513 - val_cls_loss: 0.1597 - val_box_loss: 0.0018 - val_reg_l2_loss: 0.1063 - val_loss: 0.3576
Epoch 4/50
22

## Testeo de la IA y su performance

In [ ]:
model.evaluate(test_data)

## Salvo el modelo

In [ ]:
os.mkdir('logs')

In [ ]:
model.export(export_dir='logs')

In [ ]:
model.evaluate_tflite('logs/model.tflite', test_data)

## Quantization

In [ ]:
config = QuantizationConfig.for_float16()

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('logs/model1.tflite')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_quant_model = converter.convert()

OSError: ignored

## Deteccion

### Deteccion **simple** fotos

In [ ]:
base_opt = BaseOptions(
    'logs/model1.tflite'
)

In [ ]:
detection_opt = DetectionOptions(
    score_threshold = 0.3, 
)

In [ ]:
options = ObjectDetectorOptions(
    base_opt,
    detection_opt
)

In [ ]:
detector = ObjectDetector.create_from_options(
    options
)

In [ ]:
detector = vision.ObjectDetector.create_from_file('logs/model1.tflite')

In [ ]:
tensor_image = vision.TensorImage.create_from_file('IBODS_LBL-1/test/60_jpg.rf.af5b4d9b99351329bad5223344640506.jpg')

In [ ]:
detector.detect(tensor_image)

DetectionResult(detections=[Detection(bounding_box=BoundingBox(origin_x=63, origin_y=44, width=102, height=76), categories=[Category(index=1, score=0.9296875, display_name='', category_name='autos')]), Detection(bounding_box=BoundingBox(origin_x=43, origin_y=179, width=371, height=196), categories=[Category(index=4, score=0.85546875, display_name='', category_name='pozos')]), Detection(bounding_box=BoundingBox(origin_x=158, origin_y=61, width=67, height=55), categories=[Category(index=1, score=0.765625, display_name='', category_name='autos')]), Detection(bounding_box=BoundingBox(origin_x=240, origin_y=78, width=43, height=36), categories=[Category(index=1, score=0.6875, display_name='', category_name='autos')])])

In [ ]:
results = detector.detect(tensor_image)

In [ ]:
resultsdetect = results.detections

In [ ]:
resultsdetect[0]

Detection(bounding_box=BoundingBox(origin_x=63, origin_y=44, width=102, height=76), categories=[Category(index=1, score=0.9296875, display_name='', category_name='autos')])

In [ ]:
def unique(list1):

    # initialize a null list
    unique_list = []
  
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    # print list
    return unique_list

In [ ]:
cwd = os.getcwd()

image_path = 'IBODS_LBL-1/test/60_jpg.rf.af5b4d9b99351329bad5223344640506.jpg'
model_path = 'logs/model1.tflite'
threshold = 0.3

multi_detected_objects = [] 

tensor_image = vision.TensorImage.create_from_file(image_path)
image = Image.open(image_path)
image = np.asarray(image)
wid = image.shape[1]
left = wid/3
center = left * 2
right = wid 

base_opt = BaseOptions(model_path)
detection_opt = DetectionOptions(score_threshold = threshold)
options = ObjectDetectorOptions(base_opt, detection_opt)

detector = ObjectDetector.create_from_options(options)

results = detector.detect(tensor_image)

resultsdetect = results.detections

_TEXT_COLOR = (0, 0, 255)

w_list = []

for obj in resultsdetect:
  wa = [obj.bounding_box.width]
  w_list.append(wa)
  start_point = obj.bounding_box.origin_x, obj.bounding_box.origin_y 
  end_point = obj.bounding_box.origin_x + obj.bounding_box.width, obj.bounding_box.origin_y + obj.bounding_box.height
  detect_img = cv2.rectangle(image, start_point, end_point, _TEXT_COLOR, 3)
  x1 = obj.bounding_box.origin_x
  x2 = obj.bounding_box.origin_x + obj.bounding_box.width
  middle = obj.bounding_box.width / 2
  Center = x2 - middle
  x = (x1, x2)
  if Center < left:
    position = "izquierda"
  elif Center < center:
    position = "centro"
  else:
    position = "derecha"
  categories = obj.categories
  for category in categories:
    names = category.category_name
    multi_detected_objects.append(names)
    scores = category.score * 100
  count_objs = {i:multi_detected_objects.count(i) for i in multi_detected_objects}
  contador = str(count_objs)
  final_contador = contador.strip("{ }")
  count = dict(Counter(multi_detected_objects))
  percentages = [float(scores)]
  for percentage in percentages:
    confidence = str(int(percentage)) + '%'
    label = names + ", " + confidence + ", " + position
  cv2.putText(image, label, (obj.bounding_box.origin_x,obj.bounding_box.origin_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
  cv2.putText(image, final_contador,(right - 200, right - 10) , cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1)


detect_objs = unique(multi_detected_objects)
print(w_list)
print(multi_detected_objects)
print(detect_objs)
print(count_objs)
print(count)
print(contador)
img = Image.fromarray(detect_img, 'RGB')
img.save(f'{cwd}/result/output.png')
img.show()

[[102], [371], [67], [43]]
['autos', 'pozos', 'autos', 'autos']
['autos', 'pozos']
{'autos': 3, 'pozos': 1}
{'autos': 3, 'pozos': 1}
{'autos': 3, 'pozos': 1}


In [ ]:
def drawBoundingBox(image_path, model_path, threshold):

  # Agarro el directorio (carpeta) actual
  cwd = os.getcwd()

  # Le asigno los tres parametros:
  # - La ruta de la imagen a detectar
  # - La ruta del modelo de IA
  # - El umbral de la IA

  image_path = image_path
  model_path = model_path
  threshold = threshold
  classes = ['cordon',
 'autos',
 'personas',
 'cruces',
 'pozos',
 'parar',
 'cruzar',
 'bicicleta',
 'moto',
 'escalones']

  # Lista de todos los objetos detectados y lista de todos los width de los objetos

  multi_detected_objects = []
  list_width = []
  list_heights = []

  # Leo la imagen como un tensor
  tensor_image = vision.TensorImage.create_from_file(image_path)

  # Abro y leo la imagen como un array
  image = Image.open(image_path)
  image = np.asarray(image)

  # Divido la imagen en 3 (Izquierda, derecha y centro) utilizando el ancho de la imagen

  wid = image.shape[1]
  left = wid/3
  center = left * 2
  right = wid 

  # Asigno las opciones de configuracion del modelo de deteccion (Que modelo, el umbral, etc)
  base_opt = BaseOptions(model_path)
  detection_opt = DetectionOptions(score_threshold = threshold)
  options = ObjectDetectorOptions(base_opt, detection_opt)

  # Creo el modelo de deteccion
  detector = ObjectDetector.create_from_options(options)

  # Agarro los resultados de la deteccion
  results = detector.detect(tensor_image)
  resultsdetect = results.detections

  # Creo un randomizador de colores por clase
  CLASSES = classes
  COLORS = np.random.randint(0, 255, size=(len(CLASSES), 3), dtype=np.uint8)

  # Si no detecto ningun objeto, devolver la imagen original y un mensaje

  if len(resultsdetect) == 0:
    print("Ningun objeto detectado")
    return image
  

  # En caso de si que reconozca objetos, ejecutamos el siguiente codigo

  else:

    # Recorro cada objetos de los resultdos de deteccion

    for obj in resultsdetect:

    # Selecciono todos los ids, nombres, porcentajes de los objetos detectados

      categories = obj.categories

      for category in categories: 
        names = category.category_name
        multi_detected_objects.append(names) 
        scores = category.score * 100 
        class_id = category.index

      # Cuento la cantidad de objetos que se repiten por clase

      count_objs = {i:multi_detected_objects.count(i) for i in multi_detected_objects} 
      contador = str(count_objs)
      final_contador = contador.strip("{ }")

      # Agarro los width de todos los objetos y los meto en una lista

      widths = obj.bounding_box.width
      list_width.append(widths)

      # Recorro ambas listas simultaneamente, creando una tupla con los valores de la misma posicion (El 0 de uno con el 0 del otro)
      
      class_width_list = [(multi_detected_objects[i], list_width[i]) for i in range(0, len(list_width))] 

      # Agarro los heights de todos los objetos y los meto en una lista

      heights = obj.bounding_box.height
      list_heights.append(heights)

      # Recorro ambas listas simultaneamente, creando una tupla con los valores de la misma posicion (El 0 de uno con el 0 del otro)

      class_height_list = [(multi_detected_objects[i], list_heights[i]) for i in range(0, len(list_heights))] 

      # Creo a porcentajes en una lista

      percentages = [float(scores)]

      # Por cada id (identificador unico del objeto), le asigno un color

      color = [int(c) for c in COLORS[class_id]]

      # Usando las coordenadas del punto de abajo a la izquierda, creo el punto de arriba de la derecha mediante el width y el height, y creo la bounding box, uniendo ambos puntos

      start_point = obj.bounding_box.origin_x, obj.bounding_box.origin_y 
      end_point = obj.bounding_box.origin_x + obj.bounding_box.width, obj.bounding_box.origin_y + obj.bounding_box.height
      detect_img = cv2.rectangle(image, start_point, end_point, color, 2)

      # Creo las coordenadas de los puntos en x, los de abajo

      x1 = obj.bounding_box.origin_x
      x2 = obj.bounding_box.origin_x + obj.bounding_box.width

      # Calculo para saber la posicion del objeto
      middle = obj.bounding_box.width / 2
      Center = x2 - middle

      x = (x1, x2)

      # Comprobacion de la posicion del objeto
      if Center < left:
        position = "izquierda"
      elif Center < center:
        position = "centro"
      else:
        position = "derecha"
      
      # Creo el texto a poner y lo pongo arriba de las bounding boxes

      for percentage in percentages:
        confidence = str(int(percentage)) + '%'
        label = names + ", " + confidence
        cv2.putText(image, label, (obj.bounding_box.origin_x,obj.bounding_box.origin_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        cv2.putText(image, position, (obj.bounding_box.origin_x,obj.bounding_box.origin_y - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        cv2.putText(image, final_contador,(right - 200, right - 10) , cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Leo devuelta la imagen, para pasarla de array a imagen y luego la devuelvo
    detect_objs = unique(multi_detected_objects) 
    img = Image.fromarray(detect_img, 'RGB')
    return img, detect_objs, count_objs, class_width_list, class_height_list

In [ ]:
cwd = os.getcwd()
os.mkdir(f'{cwd}/result')

In [ ]:
img, objs, contador, num_width, num_height = drawBoundingBox('IBODS_LBL-1/test/60_jpg.rf.af5b4d9b99351329bad5223344640506.jpg','logs/model1.tflite', 0.3)
img.save(f'{cwd}/result/output.png')
print(objs, contador, num_width, num_height)

['autos', 'pozos'] {'autos': 3, 'pozos': 1} [('autos', 102), ('pozos', 364), ('autos', 67), ('autos', 43)] [('autos', 76), ('pozos', 196), ('autos', 54), ('autos', 36)]


### Deteccion **compleja** fotos

In [ ]:
cwd = os.getcwd()
os.mkdir(f'{cwd}/result')

In [ ]:
cwd = os.getcwd()

DETECTION_THRESHOLD = 0.3

# Change the test file path to your test image
INPUT_IMAGE_PATH = 'IBODS_LBL-1/test/00--91-_png_jpg.rf.d3502e507f2bb074188f809e480b2fdb.jpg'

im = Image.open(INPUT_IMAGE_PATH)
im.thumbnail((512, 512), Image.ANTIALIAS)
im.save(f'{cwd}/result/input.png','PNG')

# Load the TFLite model
model_path = 'logs/model1.tflite'
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Run inference and draw detection result on the local copy of the original file
detection_result_image = run_odt_and_draw_results(
    INPUT_IMAGE_PATH,
    interpreter,
    threshold=DETECTION_THRESHOLD
)

# Show the detection result
img = Image.fromarray(detection_result_image)
img.save(f'{cwd}/result/output.png')

### Deteccion en video

In [ ]:
import cv2

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("No se puede abrir la camara")
    exit()

while True:
  ret,frame = cap.read() # Voy agarrando los frames

  if ret == True: # Si leyo correctamente los frames, signfica que el retun (ret) es true, por lo que ejecuto el codigo en ese caso
    cv2.cvtColor(frame, cv2.COLOR_BGRRGB) # Convierto todos los frames al espectro RGB
    tensor_image = vision.TensorImage.create_from_array(frame) # Convierto todos los frames a tensores
    drawBoundingBox(tensor_image, 'logs/model1.tflite', 0.3, labels)
    key = cv2.waitKey(1)
    if key == ord('q'):
      break
  if not ret:
        print("No puedo recibir imagenes (finalizaste la trasmision?). Saliendo ...")
        break
cap.release()
cv2.destroyAllWindows()

## Codigo Viejo o Extra

In [ ]:
def drawBox(image_path, model_path, threshold):

  image = cv2.imread(image_path)

  base_opt = BaseOptions(model_path)
  detection_opt = DetectionOptions(score_threshold = threshold)
  options = ObjectDetectorOptions(base_opt, detection_opt)

  detector = ObjectDetector.create_from_options(options)

  results = detector.detect(image)

  resultsdetect = results.detections

  _TEXT_COLOR = (0, 0, 255)

  for obj in resultsdetect:
    start_point = obj.bounding_box.origin_x, obj.bounding_box.origin_y 
    end_point = obj.bounding_box.origin_x + obj.bounding_box.width, obj.bounding_box.origin_y + obj.bounding_box.height
  detect_img = cv2.rectangle(image, start_point, end_point, _TEXT_COLOR, 3)
   
  return detect_img

In [ ]:
for obj in resultsdetect:
  width = [obj.bounding_box.width]
  for wid in width:
    print(wid)


102
371
67
43


In [ ]:
for obj in resultsdetect:
  categories = obj.categories
  for category in categories:
    names = category.category_name
    print(names)

autos
pozos
autos
autos


In [ ]:
for obj in resultsdetect:
  categories = obj.categories
  for category in categories:
    names = category.category_name
    scores = category.score * 100
  percentages = [float(scores)]
  for percentage in percentages:
    confidence = str(int(percentage)) + '%'
    label = names + ", " + confidence
    print(label)

autos, 92%
pozos, 85%
autos, 76%
autos, 68%
